In [2]:
import pandas as pd
import numpy as np
import requests
from requests import Request
import json
from datetime import datetime, timedelta, timezone
import time
import hmac
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from unsync import unsync
import csv

from web3 import Web3

url = 'https://mainnet.infura.io/v3/596f330c1be84362996376bbcaeebe17'
#'717e1093b1b846f196a01dc66b2432f8'
# 'f8fbb2939ecf4435a8bd442de73ee64d'

w3 = Web3(Web3.HTTPProvider(url))
from hexbytes import HexBytes

pd.options.display.float_format = '{:,.2f}'.format
w3.is_connected()

True

In [3]:
ethplorer_key = 'EK-vVxt1-VCHBYmw-LUoAS'
#'EK-7Z5Pj-6qFHsmU-odq77'
etherscan_key = 'ZG8DBBDRBNA6IKARI6DBMN76Q1F4CABS27'

In [11]:
@unsync
def is_contract(vaddr):
#check address is contract
    code = w3.eth.get_code(Web3.to_checksum_address(vaddr.lower()))
        #Web3.to_checksum_address(vaddr))
    # print(f'Code: {code}')
    if len(code.hex()) == 2:
        return False
    else:
        return True

# @unsync
def parse_holders_contracts(df):        
        buf = []
        tasks = [(addr, is_contract(addr)) for addr in df['address']]
        for addr, task in tasks:
                    contract: bool = task.result()  # type: ignore
                    # print(f'Contract: {contract}')
                    buf.append({"address": addr, "contract": contract})
        return buf

In [56]:
lsd = pd.DataFrame(pd.read_csv('lsts_list.csv'))
lsd

,minted_token_address,minted_token,main_token_address,main_token,token_order,project,chain
0,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,steth,0x00,eth,0,lido,ethereum
1,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wsteth,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,steth,1,lido,ethereum
2,0x06325440d014e39736583c165c2963ba99faf14e,stecrv,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,steth,1,curve,ethereum
3,0x182b723a58739a9c974cfdb385ceadb237453c28,stecrv_gauge,0x06325440d014e39736583c165c2963ba99faf14e,stecrv,2,curve,ethereum
4,0x828b154032950c8ff7cf8085d841723db2696056,stetheth_c_f,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,steth,1,curve,ethereum
...,...,...,...,...,...,...,...
134,0x5cc2C42BB9Ff433eEe306FfCe898C2de7866A0cd,bnwstETH,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,wsteth,2,Bancor,ethereum
135,0x21E27a5E5513D6e65C4f830167390997aA84843a,stETH-ng-f,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,steth,2,curve,ethereum
136,0x79F21BC30632cd40d2aF8134B469a0EB4C9574AA,stETH-ng-f-gauge,0x21E27a5E5513D6e65C4f830167390997aA84843a,stETH-ng-f,3,curve,ethereum
137,0x1b74fa5951dfa4c03D0127011548fA79b956f63B,cvxstETH-ng-f,0x21E27a5E5513D6e65C4f830167390997aA84843a,stETH-ng-f,3,convex,ethereum


In [44]:
# holders last update 8/07/23
holders = pd.DataFrame(pd.read_csv('holders2.csv')) #'lsts_holders.csv'
holders['address'] = [x.lower().strip() for x in holders['address']]
holders

,address,namespace,category,network
0,0xf5bce5077908a1b7370b9ae04adc565ebd643966,sushiSwap,dex,ethereum
1,0x4d9629e80118082b939e3d59e69c82a2ec08b4d5,tribe_redeemer,lending,ethereum
2,0xe66cb3a0ccee18ef61c61b99fc943c2b3035036e,gearbox,lending,ethereum
3,0xc240cb2df5a2ccac50f66d4389570d648566ea7d,gearbox,lending,ethereum
4,0x25efa336886c74ea8e282ac466bdcd0199f85bb9,gnosis_safe,multisig,ethereum
...,...,...,...,...
410,0xf48aeffdb19be3753c3185088c869a95db5a3edc,unknown4,others,ethereum
411,0xebfe63ba0264ad639b3c41d2bfe1ad708f683bc8,kyber,liquidity_pool,ethereum
412,0xd340b57aacdd10f96fc1cf10e15921936f41e29c,uniswap_v3,liquidity_pool,ethereum
413,0xd2dc4192737340b4f773ca60771b2b70c032c674,gnosis_safe,multisig,ethereum


In [73]:
# get token holders addresses, clean them for a contract addresses, compare with the existing list of holders and mark unknown addresses as TBD

all_holders = pd.DataFrame() #accumulator

for token_address in lsd.minted_token_address:
    # print(token_address)
    url = f'https://api.ethplorer.io/getTopTokenHolders/{token_address}?apiKey={ethplorer_key}&limit=500'
    response = requests.get(url)
    # print(json.loads(response.text)['holders'])
    df = pd.DataFrame(json.loads(response.text)['holders'])
    tasks = [parse_holders_contracts(df)]
    parts = [pd.DataFrame(task) for task in tasks]
    
    for part in parts:
        df = df.merge(part, on="address", how="left")
    df['balance'] = df['balance']/pow(10,18)
    df = df.query('contract == True').sort_values(by = 'balance', ascending = False) # to filter for contracts    
    df['prc'] = df['balance'].cumsum()/df['balance'].sum() # to create a column prc 
    df = df.query('prc <= 0.97').sort_values('prc', ascending=False) # to filter for prc
    df = df.drop(['balance', 'share', 'contract', 'prc'], axis='columns') # to drop unuseful columns
    all_holders = all_holders.append(df, sort=False) # to concat all dfs
all_holders = all_holders.merge(holders, on='address', how = 'outer') # to get the contracts names
all_holders = all_holders.fillna('TBD') # to replace NaN value with TBD
all_holders.to_csv(f'updated_holders.csv',index=False) #save the list of all_holders
# print(all_holders)
    
    

/var/folders/gg/qxh75fss5kx034ck3tcvg10r0000gp/T/ipykernel_1115/3153575395.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_holders = all_holders.append(df, sort=False) # concat all dfs
/var/folders/gg/qxh75fss5kx034ck3tcvg10r0000gp/T/ipykernel_1115/3153575395.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_holders = all_holders.append(df, sort=False) # concat all dfs
/var/folders/gg/qxh75fss5kx034ck3tcvg10r0000gp/T/ipykernel_1115/3153575395.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_holders = all_holders.append(df, sort=False) # concat all dfs
/var/folders/gg/qxh75fss5kx034ck3tcvg10r0000gp/T/ipykernel_1115/3153575395.py:20: FutureWarning: The frame.append method is deprecated and will be remo

In [74]:
unknown_addr = all_holders.loc[all_holders['namespace'] == 'TBD'] # get unknown addresses from all_holders list
unknown_addr

,address,namespace,category,network
22,0x889edc2edab5f40e902b864ad4d7ade8e412f9b1,TBD,TBD,TBD
147,0xa57b8d98dae62b26ec3bcc4a365338157060b234,TBD,TBD,TBD
166,0x1c9798db3773d30780ff6448e015e7775c1ae75b,TBD,TBD,TBD


In [75]:
# check unknown addresses for token_contract
list = [] #accumulator
for address in unknown_addr['address']:
    # print(address)
    url = f'https://api.ethplorer.io/getTopTokenHolders/{address}?apiKey={ethplorer_key}&limit=500'
    response = requests.get(url)
    # print(json.loads(response.text))
    if response.status_code == 200:
        # print(response)
        list= np.append(list,address)
    else: 
        continue # print(address, 'is not token contract')
print(list)

[]


In [76]:
# add list of new token addresses to existing list of lsts
list = pd.DataFrame(list, columns=['minted_token_address']) # index=('minted_token_address'),
lsd = lsd.append(list, ignore_index=True)
lsd = lsd.fillna('TBD')
lsd.to_csv(f'updated_lsts_list.csv') #save updated list of LSTs 

/var/folders/gg/qxh75fss5kx034ck3tcvg10r0000gp/T/ipykernel_1115/1006773965.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lsd = lsd.append(list, ignore_index=True)
